# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
collection_names = db.list_collection_names()
print("Collections in the 'uk_food' database:")
for name in collection_names:
    print(name)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
print("First establishment with a hygiene score of 20:")
pprint(first_document)

In [ ]:
# Convert the result to a Pandas DataFrame
documents = establishments.find(query)
df = pd.DataFrame(list(documents))

print("DataFrame created from the query result:")
print(df)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with a RatingValue of 4 or higher: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
print("First establishment in London with a RatingValue of 4 or higher:")
pprint(first_document)


In [ ]:
# Convert the result to a Pandas DataFrame
general_query = {}  # An empty query matches all documents
documents = establishments.find(general_query).limit(10)
df = pd.DataFrame(list(documents))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
degree_search = 0.01
latitude = 0.01
longitude = 0.01

latitude = float(latitude)
longitude = float(longitude)

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}
sort =  sort = [("scores.Hygiene", 1)]  #ascending order

# Print
print(df)

In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
pipeline = [
    # 1. Matches establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    
   # 2. Groups the matches by Local Authority
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    
    # 3. Sorts the matches from highest to lowest
    {"$sort": {"count": -1}}
]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of documents in the result: {len(results)}")

# Print the first 10 results
print("First 10 results:")
for result in results[:10]:
    print(result)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))
